 
# Установка библиотек

In [148]:
!pip install ultralytics
!pip install clearml
!pip install supervision

In [149]:
from ultralytics import YOLO
import cv2
import numpy as np
import supervision as sv

In [199]:
model = YOLO("yolov8n-pose.pt")

 
# Общие геометрические функции, которые потребуются для написания эвристик

![](https://github.com/ybelonogov/ml_task/tree/master/data/keypoints_yolo.png)

In [152]:
def calculate_distance(keypoints, idx1, idx2):
    x1, y1 = keypoints.xy[0][idx1]
    x2, y2 = keypoints.xy[0][idx2]
    if keypoints.has_visible:
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        return distance
    else:
        return None


def choose_side(keypoints):
    r_shoulder = calculate_distance(keypoints, 6, 8)
    l_shoulder = calculate_distance(keypoints, 5, 7)
    return r_shoulder > l_shoulder

In [153]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle 

 
# Эвристики для отжиманий

In [181]:
def check_position_body_line(keypoints):
    if side:
        #распремлена поясница
        angle1 = calculate_angle(keypoints.xy[0][6],
                                 keypoints.xy[0][12],
                                 keypoints.xy[0][14], )
        #распрямлены ноги
        angle2 = calculate_angle(keypoints.xy[0][12],
                                 keypoints.xy[0][14],
                                 keypoints.xy[0][16], )
        #изгиб локтя
        angle3 = calculate_angle(keypoints.xy[0][6],
                                 keypoints.xy[0][8],
                                 keypoints.xy[0][10], )
    else:
        angle1 = calculate_angle(keypoints.xy[0][5],
                                 keypoints.xy[0][11],
                                 keypoints.xy[0][13], )
        angle2 = calculate_angle(keypoints.xy[0][11],
                                 keypoints.xy[0][13],
                                 keypoints.xy[0][15], )
        angle3 = calculate_angle(keypoints.xy[0][5],
                                 keypoints.xy[0][7],
                                 keypoints.xy[0][9], )

    # return angle1 > 120 and (angle2 > 120 or angle2 < 45) and angle3 < 120
    return angle1 + angle2 > 260 and angle3 < 120


def get_push_up_angle(keypoints):
    if side:
        angle = calculate_angle(keypoints.xy[0][6],
                                keypoints.xy[0][8],
                                keypoints.xy[0][12],
                                ) - calculate_angle(keypoints.xy[0][6],
                                                    keypoints.xy[0][8],
                                                    keypoints.xy[0][10], )
    else:
        angle = calculate_angle(keypoints.xy[0][5],
                                keypoints.xy[0][7],
                                keypoints.xy[0][11],
                                ) - calculate_angle(keypoints.xy[0][5],
                                                    keypoints.xy[0][7],
                                                    keypoints.xy[0][9], )
    return angle


def check_push_up_down(keypoints, prev_angle):
    angle = get_push_up_angle(keypoints=result.keypoints)
    if angle >= prev_angle:
        # prev_angle = max(angle, prev_angle)
        return True
    else:

        return False


def check_push_up(keypoints, prev_angle):
    angle = get_push_up_angle(keypoints=result.keypoints)
    if angle < prev_angle:
        return True
    else:
        return False


 
# Эвристики для приседаний

In [188]:
def check_position_legs_bent(keypoints):
    if side:
        #распремлена поясница
        angle1 = calculate_angle(keypoints.xy[0][6],
                                 keypoints.xy[0][12],
                                 keypoints.xy[0][14], )
        #распрямлены ноги
        angle2 = calculate_angle(keypoints.xy[0][12],
                                 keypoints.xy[0][14],
                                 keypoints.xy[0][16], )
    else:
        angle1 = calculate_angle(keypoints.xy[0][5],
                                 keypoints.xy[0][11],
                                 keypoints.xy[0][13], )
        angle2 = calculate_angle(keypoints.xy[0][11],
                                 keypoints.xy[0][13],
                                 keypoints.xy[0][15], )
    return angle1 + angle2 < 260


def get_squats_angls(keypoints):
    if side:
        # колено
        angle1 = calculate_angle(keypoints.xy[0][12],
                                 keypoints.xy[0][14],
                                 keypoints.xy[0][16], )
        # таз
        angle2 = calculate_angle(keypoints.xy[0][6],
                                 keypoints.xy[0][12],
                                 keypoints.xy[0][14], )
    else:
        angle1 = calculate_angle(keypoints.xy[0][11],
                                 keypoints.xy[0][13],
                                 keypoints.xy[0][15], )
        angle2 = calculate_angle(keypoints.xy[0][5],
                                 keypoints.xy[0][11],
                                 keypoints.xy[0][13], )
    return angle1, angle2


def check_squats_down(keypoints, prev_angls):
    angls = get_squats_angls(keypoints)
    if angls[0] + angls[1] > prev_angls[1] + prev_angls[0]:
        return True
    return False


def check_squats_up(keypoints, prev_angls):
    angls = get_squats_angls(keypoints)
    if angls[0] + angls[1] < prev_angls[1] + prev_angls[0]:
        return True
    return False

 
# TODO: Эвристики для подтягиваний
## Делать бесполезно, так-как нужно обучать класификатор   

In [156]:
def check_pull_ups(keypoints, movment_down):
    return 

# Вывод основной + debug информации на экран

In [194]:
def print_count(image, pull_ups, push_ups, squats, person):
    org = [20, 100]  # Например, (x, y) = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 4  # Масштаб шрифта
    font_color = (0, 200, 0)  # Цвет шрифта в формате BGR   
    thickness = 6  # Толщина шрифта
    org[1] = org[1] + (person - 1) * 400
    # text = "person: {0} ".format(person)
    # cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    # org[1] += 100
    text = "pull ups: {0} ".format(pull_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "push ups: {0} ".format(push_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "squats: {0}".format(squats)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)


def print_debag_info(image, movment_down, prev_angle, position_line, pos_squat):
    org = [20, 100]  # Например, (x, y) = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2  # Масштаб шрифта
    font_color = (0, 200, 0)  # Цвет шрифта в формате BGR   
    thickness = 2  # Толщина шрифта
    org[1] = org[1] + 400
    # text = "{0} ".format(push_ups)
    # cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    # org[1] += 100
    text = "movment_down: {0} ".format(movment_down)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 50
    text = "prev_angle: {0} ".format(prev_angle)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 50
    text = "pos= line: {0}".format(position_line)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 50
    text = "pos= squat: {0}".format(pos_squat)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)

# Загрузка и обработка видео  

In [211]:
video = cv2.VideoCapture("data/push_up4.mp4")
video_info = sv.VideoInfo.from_video_path("data/push_up4.mp4")
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter("ans/output.mp4", fourcc, video_info.fps,
                               (video_info.width, video_info.height))
# result = model(source="data/push_ups1.mp4", show=False, conf=0.3, save=True)
push_up_count = 0
pull_up_count = 0
squats_count = 0
movment_down = True
prev_angle = 0
cadr_num = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if cadr_num % 5 == 0:
        results = model.predict(frame)
        person = 1
        # for result in results:
        result = results[0]
        side = choose_side(keypoints=result.keypoints)
        # push_ups подсчёт 
        if check_position_body_line(result.keypoints) and not check_position_legs_bent(result.keypoints):

            if movment_down and check_push_up_down(result.keypoints, prev_angle):
                movment_down = False
            if not movment_down and check_push_up(result.keypoints, prev_angle):
                push_up_count += 1
                movment_down = True
        prev_angle = get_push_up_angle(result.keypoints)
        # print_count(frame, pull_up_count, push_up_count, squats_count, person)
        # print_debag_info(frame, movment_down_squats, prev_angls[0] ,
        #                  check_position_body_line(result.keypoints),
        #                  check_position_legs_bent(result.keypoints))
        # output_video.write(frame)

    cadr_num += 1
    print_count(frame, pull_up_count, push_up_count, squats_count, person)
    output_video.write(frame)

video.release()
output_video.release()
cv2.destroyAllWindows()

In [221]:
video = cv2.VideoCapture("data/squats.mp4")
# output_path = os.path.join("", "output2.mp4")
video_info = sv.VideoInfo.from_video_path("data/squats.mp4")
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter("output5.mp4", fourcc, video_info.fps,
                               (video_info.width, video_info.height))
# result = model(source="data/push_ups1.mp4", show=False, conf=0.3, save=True)
push_up_count = 0
pull_up_count = 0
squats_count = 0
movment_down = True
movment_down_squats = True
prev_angls = (0,0)
cadr_num = 0
while True:
    ret, frame = video.read()
    if not ret:
        break
    if cadr_num % 5 == 0:
        results = model.predict(frame)
        person = 1
        # for result in results:
        result = results[0]
        side = choose_side(keypoints=result.keypoints)

        # squats подчёт
        if check_position_legs_bent(result.keypoints):
            if movment_down_squats and check_squats_down(result.keypoints, prev_angls):
                movment_down_squats = False
            if not movment_down_squats and check_squats_up(result.keypoints, prev_angls):
                movment_down_squats = True
                squats_count += 1
        prev_angls = get_squats_angls(result.keypoints)

        # print_count(frame, pull_up_count, push_up_count, squats_count, person)
        # print_debag_info(frame, movment_down_squats, prev_angls[0] ,
        #                  check_position_body_line(result.keypoints),
        #                  check_position_legs_bent(result.keypoints))
        # output_video.write(frame)

    cadr_num += 1
    print_count(frame, pull_up_count, push_up_count, squats_count, person)
    output_video.write(frame)

video.release()
output_video.release()
cv2.destroyAllWindows()

In [222]:
squats_count

3